In [29]:
from dataloader_ds import AudioDataset, load_wav
from tqdm import tqdm
from torch.utils.data import DataLoader
from models import Conformer_SER_SSL, SSL_SER_Config, Downstream_model
from torch.nn import functional as F
import torch.nn as nn
import torch
import json
import argparse
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

device = 'cuda'


batches = [16, 32, 64, 128, 256, 512]
for xyz in batches:
    #remove _200 to get 100 epochs performance
    load_dir = "./logs/log_" + str(xyz) + "/" + "checkpoint_200.pth"
    dev = pd.read_csv("train.csv")
    device = 'cuda'
    with open("config.json", "r", encoding='utf8') as f:
        cfg_dict = json.load(f)
    cfg = SSL_SER_Config(cfg_dict)
    ssl = Conformer_SER_SSL(cfg).to(device)
    ssl.load_state_dict(torch.load(load_dir))

    labels = []
    embedding = []
    with torch.no_grad():
        for point in tqdm(range(len(dev))):
            path = dev['Path'][point]
            label = dev['Label'][point]
            if labels.count(label)<3:
                audio, sampling_rate = load_wav(path)
                input = ssl.get_embedding(torch.tensor(audio, device=device).unsqueeze(0))
                labels.append(label)
                embedding.append(input.detach().cpu().numpy())


    test_labels = []
    pred_labels = []
    dev = pd.read_csv("test.csv")
    with torch.no_grad():
        for point in tqdm(range(len(dev))):
            path = dev['Path'][point]
            label = dev['Label'][point]
            audio, sampling_rate = load_wav(path)
            input = ssl.get_embedding(torch.tensor(audio, device=device).unsqueeze(0)).detach().cpu().numpy()
            test_labels.append(label)
            max_cosine_similarity = -1
            pred_label = -1
            for i in range(len(embedding)):
                emb = embedding[i]
                dot_product = np.dot(input.squeeze(0), emb.squeeze(0))
                norm_tensor1 = np.linalg.norm(input.squeeze(0))
                norm_tensor2 = np.linalg.norm(emb.squeeze(0))
                cosine_similarity = dot_product / (norm_tensor1 * norm_tensor2)
                if cosine_similarity>max_cosine_similarity:
                    pred_label = labels[i]
                    max_cosine_similarity = cosine_similarity
            pred_labels.append(pred_label)

    cnt = 0
    for i in range(len(pred_labels)):
        if pred_labels[i]==test_labels[i]:
            cnt+=1
    print(xyz)
    print(cnt/len(dev))



100%|██████████| 600/600 [00:23<00:00, 25.72it/s]


16
0.7333333333333333


100%|██████████| 600/600 [00:22<00:00, 26.11it/s]


32
0.78


100%|██████████| 600/600 [00:21<00:00, 27.62it/s]


...

128
0.76


100%|██████████| 600/600 [00:21<00:00, 27.37it/s]


256
0.7516666666666667


100%|██████████| 600/600 [00:21<00:00, 27.50it/s]

512
0.7416666666666667
